# Ictonyx Example: PyTorch Classification Variability Study

This notebook trains a simple feedforward network on the Iris dataset multiple times
and reports the distribution of validation accuracy across runs.

**Requirements:** `pip install ictonyx torch scikit-learn`

In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

import ictonyx as ix
from ictonyx import (
    ModelConfig,
    PyTorchModelWrapper,
    ArraysDataHandler,
    run_variability_study,
)

print(f"Ictonyx v{ix.__version__}")
print(f"PyTorch v{torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

2026-02-16 13:41:58.141586: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-16 13:41:58.165944: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-16 13:41:58.165965: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-16 13:41:58.166806: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-16 13:41:58.171210: I tensorflow/core/platform/cpu_feature_guar

Ictonyx v0.3.2
PyTorch v2.10.0+cu128
Device: cuda


## 1. Load and Prepare Data

We use the classic Iris dataset — 150 samples, 4 features, 3 classes.
StandardScaler normalizes features, which helps the network converge faster.

In [3]:
iris = load_iris()
X = StandardScaler().fit_transform(iris.data).astype(np.float32)
y = iris.target.astype(np.int64)

data_handler = ArraysDataHandler(X, y)

print(f"Samples: {len(X)}")
print(f"Features: {X.shape[1]}")
print(f"Classes: {len(np.unique(y))}")

Samples: 150
Features: 4
Classes: 3


## 2. Define the Model Builder

The model builder is a **factory function** that creates a fresh model for each run.
This is essential — each variability study run needs its own randomly initialized model.

The `PyTorchModelWrapper` takes:
- An `nn.Module` (your network architecture)
- A loss function (`criterion`)
- An optimizer class + params (not an instance — because the optimizer must bind to each new model's parameters)

In [4]:
def create_iris_net(config: ModelConfig) -> PyTorchModelWrapper:
    """Factory function: creates a fresh model each run."""
    model = nn.Sequential(
        nn.Linear(4, 32),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(32, 16),
        nn.ReLU(),
        nn.Linear(16, 3),
    )
    return PyTorchModelWrapper(
        model,
        criterion=nn.CrossEntropyLoss(),
        optimizer_class=torch.optim.Adam,
        optimizer_params={'lr': config.get('learning_rate', 0.01)},
        task='classification',
    )

# Quick sanity check
test_wrapper = create_iris_net(ModelConfig({'learning_rate': 0.01}))
print(repr(test_wrapper))

Ictonyx BaseModelWrapper(id='', type='Sequential', is_trained=No)


## 3. Run the Variability Study

This trains the model 10 times, each with a different random initialization
but deterministic seeding (seed=42), so the study is fully reproducible.

In [5]:
config = ModelConfig({
    'epochs': 30,
    'batch_size': 16,
    'learning_rate': 0.01,
    'verbose': 0,
})

results = run_variability_study(
    model_builder=create_iris_net,
    data_handler=data_handler,
    model_config=config,
    num_runs=10,
    seed=42,
)

Loading and preparing data...
Array splits - Train: 105, Val: 15, Test: 30
Data loaded successfully

Starting Variability Study
  Runs: 10
  Epochs per run: 30
  Execution mode: in standard mode
  Seed: 42



2026-02-16 13:43:12.374755: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-16 13:43:12.376269: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-02-16 13:43:12.377454: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

--------------------------------------------------

Study Summary:
  Successful runs: 10/10
  train_loss: 0.0272 (SD = 0.0042)
  train_accuracy: 0.9914 (SD = 0.0051)
  val_loss: 0.5154 (SD = 0.1372)
  val_accuracy: 0.8667 (SD = 0.0000)


## 4. Examine Results

In [6]:
print(results.summarize())

Variability Study Results
Successful runs: 10
Seed: 42
train_accuracy:
  Mean: 0.9914
  Std:  0.0051
  Min:  0.9810
  Max:  1.0000
train_loss:
  Mean: 0.0272
  Std:  0.0042
  Min:  0.0209
  Max:  0.0363
val_accuracy:
  Mean: 0.8667
  Std:  0.0000
  Min:  0.8667
  Max:  0.8667
val_loss:
  Mean: 0.5154
  Std:  0.1372
  Min:  0.2044
  Max:  0.6920


In [7]:
print("Available metrics:", results.get_available_metrics())
print()
print("Per-run val_accuracy:")
for i, acc in enumerate(results.get_metric_values('val_accuracy'), 1):
    print(f"  Run {i}: {acc:.4f}")

Available metrics: ['train_accuracy', 'train_loss', 'val_accuracy', 'val_loss']

Per-run val_accuracy:
  Run 1: 0.8667
  Run 2: 0.8667
  Run 3: 0.8667
  Run 4: 0.8667
  Run 5: 0.8667
  Run 6: 0.8667
  Run 7: 0.8667
  Run 8: 0.8667
  Run 9: 0.8667
  Run 10: 0.8667


In [8]:
# Summary DataFrame — one row per run, all final-epoch metrics
results.to_dataframe()

,run_id,final_train_loss,final_train_accuracy,final_val_loss,final_val_accuracy,test_loss,test_accuracy
0,1,0.030083,0.980952,0.409666,0.866667,0.075551,0.966667
1,2,0.036283,0.990476,0.204389,0.866667,0.049065,0.966667
2,3,0.030850,0.990476,0.546875,0.866667,0.085400,0.966667
3,4,0.026064,0.990476,0.575677,0.866667,0.131469,0.966667
4,5,0.025185,0.990476,0.516833,0.866667,0.089837,0.966667
5,6,0.028751,0.990476,0.542759,0.866667,0.124648,0.966667
6,7,0.020906,1.000000,0.591200,0.866667,0.088215,0.966667
7,8,0.026816,1.000000,0.692049,0.866667,0.103308,0.966667
8,9,0.023907,0.990476,0.673272,0.866667,0.170302,0.966667
9,10,0.022959,0.990476,0.401118,0.866667,0.042196,0.966667
